In [7]:
hdu1 = fits.open('HGCA_vDR2_withchisq.fits')
hdu1.info()

Filename: HGCA_vDR2_withchisq.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU       4   ()      
  1                1 BinTableHDU     84   115663R x 38C   [J, K, D, D, E, E, 8A, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, D, D, D, D, E, E, E, E, E, E, E, E, E, E]   


In [14]:
hgca = hdu1[1]
header = hgca.header
data = hgca.data
cols = hdu1[1].columns
header

XTENSION= 'BINTABLE'           / binary table extension                         
BITPIX  =                    8 / array data type                                
NAXIS   =                    2 / number of array dimensions                     
NAXIS1  =                  184 / length of dimension 1                          
NAXIS2  =               115663 / length of dimension 2                          
PCOUNT  =                    0 / number of group parameters                     
GCOUNT  =                    1 / number of groups                               
TFIELDS =                   38 / number of table fields                         
TTYPE1  = 'hip_id  '                                                            
TFORM1  = 'J       '                                                            
TTYPE2  = 'gaia_source_id'                                                      
TFORM2  = 'K       '                                                            
TTYPE3  = 'gaia_ra '        

In [22]:
print(len(data))
print(np.where(data['chisq']>12)[0].shape[0]/len(data))

115663
0.24350051442552934


In [114]:
highacceltargets = data[np.where(data['chisq']>12)[0]]
print('HipID',highacceltargets[0][0])
print('GaiaID',highacceltargets[0][1])
print('chisq',highacceltargets[0][-1])


HipID 1
GaiaID 2738327528519591936
chisq 894.38995


In [44]:
from myastrotools.gaia_tools import get_RUWE
get_RUWE(2738327528519591936)

For 2738327528519591936 RUWE= 0.986564326072574


In [151]:
# Get Rizzuto 2011 membership:
hdu1 = fits.open('J_MNRAS_416_3108_tablea1.dat.fits')
sco = hdu1[1].data
scoheader = hdu1[1].header

scocen = pd.DataFrame(columns=['HipID','MembershipProb'])
# Place into pandas dataframe:
for i in range(sco.shape[0]):
    #dfObj = dfObj.append({'User_ID': 23, 'UserName': 'Riti', 'Action': 'Login'}, ignore_index=True)
    scocen = scocen.append({'HipID':sco[i][0], 'MembershipProb':sco[i][1]}, ignore_index=True)
# Sort by highest membership probability
scocen = scocen.sort_values('MembershipProb',ascending=False)
scocen = scocen.reset_index(drop=True)

In [157]:
#for i in range(highacceltargets.shape[0]):

scocenind = np.array([], dtype='int64')
hip = np.array([], dtype='int64')
gaia = np.array([], dtype='int64')
chisq = np.array([])

# Pull out ScoCen targets from high accel list:
for i in range(highacceltargets.shape[0]):
    ind = np.where(scocen['HipID'] == highacceltargets[i][0])
    if len(ind[0]) != 0:
        hip = np.append(hip,int(highacceltargets[i][0]))
        gaia = np.append(gaia,int(highacceltargets[i][1]))
        chisq = np.append(chisq,int(highacceltargets[i][-1]))
        scocenind = np.append(scocenind,ind)

hip

array([53701, 55334, 55425, 55899, 56963, 57710, 57947, 58146, 58283,
       58416, 58884, 58899, 58901, 59119, 59173, 59449, 59505, 59603,
       60009, 60459, 60851, 60855, 61049, 61087, 62026, 62032, 62171,
       62428, 62683, 63005, 63041, 63204, 64053, 64322, 64425, 65219,
       65271, 65822, 66454, 66651, 66908, 67199, 67919, 67957, 68532,
       68722, 69302, 70483, 70809, 70833, 70931, 71140, 71321, 71724,
       72149, 72158, 72627, 73913, 73937, 74104, 74479, 75151, 75613,
       76084, 76126, 76143, 76600, 76633, 77295, 77388, 77399, 77815,
       78196, 78233, 78416, 78494, 78549, 78581, 78754, 78821, 78847,
       78877, 78918, 79031, 79054, 79081, 79097, 79098, 79142, 79374,
       79392, 79399, 79690, 79860, 80062, 80371, 80461, 80711, 80896,
       81447, 81914, 82569])

In [159]:
scocen_accel = scocen.loc[scocenind]
scocen_accel = scocen_accel.reset_index(drop=True)
scocen_accel['GaiaID'] = gaia
scocen_accel['chisq'] = chisq
scocen_accel['RUWE'] = ""
scocen_accel['AstrExNoise'] = ""
scocen_accel['GaiaRA'] = ""
scocen_accel['GaiaDEC'] = ""
# Sort by membership probability:
scocen_accel = scocen_accel.sort_values('MembershipProb',ascending=False)
scocen_accel = scocen_accel.reset_index(drop=True)
scocen_accel

,HipID,MembershipProb,GaiaID,chisq,RUWE,AstrExNoise,GaiaRA,GaiaDEC
0,72627,93,6100404013458686592,245.0,,,,
1,71321,92,6099886349638035456,282.0,,,,
2,79054,91,6242050415668507392,12.0,,,,
3,78196,90,6039491481598054528,66.0,,,,
4,78847,89,6243323448279638528,7230.0,,,,
5,66454,89,6084002083108541952,30.0,,,,
6,67957,89,6090407734775872000,1021.0,,,,
7,78549,89,6243172677747745664,20.0,,,,
8,68532,89,6111335598580052736,605.0,,,,
9,76633,89,6253938511486847360,16.0,,,,


In [271]:
def get_stuff(source_ids):
    '''Use Gaia DR2 source id to return the re-normalised unit weight error for the solution'''
    import warnings
    warnings.filterwarnings("ignore")
    import numpy as np
    from astroquery.gaia import Gaia
    from myastrotools.gaia_tools import RUWE_u0,RUWE_u0_nocolor

    job = Gaia.launch_job("SELECT * FROM gaiadr2.gaia_source WHERE source_id = "+str(source_ids))
    j = job.get_results()
    job = Gaia.launch_job("SELECT * FROM gaiadr2.ruwe WHERE source_id = "+str(source_ids))
    k = job.get_results()
    return (k['ruwe'], j['astrometric_excess_noise'],j['ra'],j['dec'],j['phot_g_mean_mag'], j['parallax'])

In [175]:
# Get Gaia RUWE and excess noise values:
ruwe = np.array([])
noise = np.array([])
gra = np.array([])
gdec = np.array([])
gmag = np.array([])
for i in range(len(scocen_accel)):
    stuff = get_stuff(scocen_accel.loc[i]['GaiaID'])
    ruwe = np.append(ruwe,stuff[0][0])
    noise = np.append(noise,stuff[1][0])
    gra = np.append(gra,stuff[2][0])
    gdec = np.append(gdec,stuff[3][0])
    gmag = np.append(gmag,stuff[4][0])
    
scocen_accel['RUWE'] = ruwe
scocen_accel['AstrExNoise'] = noise
scocen_accel['GaiaRA'] = gra
scocen_accel['GaiaDEC'] = gdec
scocen_accel['GaiaGMag'] = gmag
scocen_accel

,HipID,MembershipProb,GaiaID,chisq,RUWE,AstrExNoise,GaiaRA,GaiaDEC,GaiaGMag
0,78581,86,6039946610694672640,9290.0,0.998880,0.000000,240.683851,-30.667244,8.920046
1,78847,89,6243323448279638528,7230.0,0.896259,0.000000,241.430737,-21.838894,7.741512
2,78877,61,6242305850261767168,3596.0,0.987633,0.316001,241.526528,-23.606418,5.868583
3,76143,51,6238804592922052224,2637.0,1.119424,0.154336,233.289544,-24.490566,6.940744
4,57947,50,5341921344586461440,2050.0,1.735048,0.247405,178.268936,-58.253518,8.031940
5,80896,86,6031818294879846656,1973.0,1.006181,0.000000,247.796012,-29.998049,8.448680
6,62026,77,6073279508288863744,1648.0,1.728195,0.505047,190.707080,-55.947042,6.030624
7,56963,81,5370222361543311744,1397.0,0.951089,0.000000,175.160539,-49.509227,7.808117
8,64322,74,5863295048178769792,1045.0,0.922026,0.000000,197.745699,-62.087787,8.126640
9,79097,66,6043732950778719744,1039.0,0.974191,0.000000,242.181895,-25.376936,8.599725


In [176]:
# Sort by chisq:
scocen_accel = scocen_accel.sort_values('chisq',ascending=False)
scocen_accel = scocen_accel.reset_index(drop=True)
scocen_accel

,HipID,MembershipProb,GaiaID,chisq,RUWE,AstrExNoise,GaiaRA,GaiaDEC,GaiaGMag
0,78581,86,6039946610694672640,9290.0,0.998880,0.000000,240.683851,-30.667244,8.920046
1,78847,89,6243323448279638528,7230.0,0.896259,0.000000,241.430737,-21.838894,7.741512
2,78877,61,6242305850261767168,3596.0,0.987633,0.316001,241.526528,-23.606418,5.868583
3,76143,51,6238804592922052224,2637.0,1.119424,0.154336,233.289544,-24.490566,6.940744
4,57947,50,5341921344586461440,2050.0,1.735048,0.247405,178.268936,-58.253518,8.031940
5,80896,86,6031818294879846656,1973.0,1.006181,0.000000,247.796012,-29.998049,8.448680
6,62026,77,6073279508288863744,1648.0,1.728195,0.505047,190.707080,-55.947042,6.030624
7,56963,81,5370222361543311744,1397.0,0.951089,0.000000,175.160539,-49.509227,7.808117
8,64322,74,5863295048178769792,1045.0,0.922026,0.000000,197.745699,-62.087787,8.126640
9,79097,66,6043732950778719744,1039.0,0.974191,0.000000,242.181895,-25.376936,8.599725


In [202]:
from astroquery.simbad import Simbad
customSimbad = Simbad()
print(customSimbad.get_votable_fields())
customSimbad.add_votable_fields('mk', 'ids', 'otype')
print(customSimbad.get_votable_fields())
print(scocen_accel['HipID'][0])
result_table = customSimbad.query_object('HIP '+str(scocen_accel['HipID'][0]))
result_table['OTYPE'][0]

['main_id', 'coordinates']
['main_id', 'coordinates', 'mk', 'ids', 'otype']
78581


b'RotV*'

In [204]:
otype = np.array([])
for i in range(len(scocen_accel)):
    result_table = customSimbad.query_object('HIP '+str(scocen_accel['HipID'][i]))
    otype = np.append(otype,result_table['OTYPE'][0])
    
scocen_accel['otype'] = otype
scocen_accel   

,HipID,MembershipProb,GaiaID,chisq,RUWE,AstrExNoise,GaiaRA,GaiaDEC,GaiaGMag,otype
0,78581,86,6039946610694672640,9290.0,0.998880,0.000000,240.683851,-30.667244,8.920046,b'RotV*'
1,78847,89,6243323448279638528,7230.0,0.896259,0.000000,241.430737,-21.838894,7.741512,b'**'
2,78877,61,6242305850261767168,3596.0,0.987633,0.316001,241.526528,-23.606418,5.868583,b'**'
3,76143,51,6238804592922052224,2637.0,1.119424,0.154336,233.289544,-24.490566,6.940744,b'**'
4,57947,50,5341921344586461440,2050.0,1.735048,0.247405,178.268936,-58.253518,8.031940,b'PM*'
5,80896,86,6031818294879846656,1973.0,1.006181,0.000000,247.796012,-29.998049,8.448680,b'Star'
6,62026,77,6073279508288863744,1648.0,1.728195,0.505047,190.707080,-55.947042,6.030624,b'Star'
7,56963,81,5370222361543311744,1397.0,0.951089,0.000000,175.160539,-49.509227,7.808117,b'Star'
8,64322,74,5863295048178769792,1045.0,0.922026,0.000000,197.745699,-62.087787,8.126640,b'Star'
9,79097,66,6043732950778719744,1039.0,0.974191,0.000000,242.181895,-25.376936,8.599725,b'RotV*'


In [277]:
ind = np.array([], dtype='int64')
for i in range(len(scocen_accel)):
    if scocen_accel['otype'][i] == b'Star':
        ind = np.append(ind, i)
scocen_accel.loc[ind]

,HipID,MembershipProb,GaiaID,chisq,RUWE,AstrExNoise,GaiaRA,GaiaDEC,GaiaGMag,otype
5,80896,86,6031818294879846656,1973.0,1.006181,0.000000,247.796012,-29.998049,8.448680,b'Star'
6,62026,77,6073279508288863744,1648.0,1.728195,0.505047,190.707080,-55.947042,6.030624,b'Star'
7,56963,81,5370222361543311744,1397.0,0.951089,0.000000,175.160539,-49.509227,7.808117,b'Star'
8,64322,74,5863295048178769792,1045.0,0.922026,0.000000,197.745699,-62.087787,8.126640,b'Star'
10,76084,87,6208562250724337152,1022.0,1.249710,0.087385,233.083793,-31.142841,8.501759,b'Star'
11,67957,89,6090407734775872000,1021.0,1.488573,0.137400,208.755192,-50.750674,9.066534,b'Star'
12,78754,73,5998514882260320640,872.0,0.997232,0.131773,241.176287,-37.816228,7.035934,b'Star'
13,55334,58,5228361962579665152,841.0,1.083729,0.000000,169.969254,-70.618413,8.038264,b'Star'
14,61087,88,6054752668651631872,800.0,0.970687,0.000000,187.802322,-61.908794,7.861641,b'Star'
15,68722,72,6120528168618891904,638.0,1.234152,0.123006,211.020420,-37.283635,8.289087,b'Star'


In [284]:
scocen_stars = scocen_accel.loc[ind]
scocen_stars = scocen_stars.reset_index(drop=True)
scocen_stars

,HipID,MembershipProb,GaiaID,chisq,RUWE,AstrExNoise,GaiaRA,GaiaDEC,GaiaGMag,otype
0,80896,86,6031818294879846656,1973.0,1.006181,0.000000,247.796012,-29.998049,8.448680,b'Star'
1,62026,77,6073279508288863744,1648.0,1.728195,0.505047,190.707080,-55.947042,6.030624,b'Star'
2,56963,81,5370222361543311744,1397.0,0.951089,0.000000,175.160539,-49.509227,7.808117,b'Star'
3,64322,74,5863295048178769792,1045.0,0.922026,0.000000,197.745699,-62.087787,8.126640,b'Star'
4,76084,87,6208562250724337152,1022.0,1.249710,0.087385,233.083793,-31.142841,8.501759,b'Star'
5,67957,89,6090407734775872000,1021.0,1.488573,0.137400,208.755192,-50.750674,9.066534,b'Star'
6,78754,73,5998514882260320640,872.0,0.997232,0.131773,241.176287,-37.816228,7.035934,b'Star'
7,55334,58,5228361962579665152,841.0,1.083729,0.000000,169.969254,-70.618413,8.038264,b'Star'
8,61087,88,6054752668651631872,800.0,0.970687,0.000000,187.802322,-61.908794,7.861641,b'Star'
9,68722,72,6120528168618891904,638.0,1.234152,0.123006,211.020420,-37.283635,8.289087,b'Star'


In [285]:
# add parallax:
plx = np.array([])
for i in range(len(scocen_stars)):
    stuff = get_stuff(scocen_stars.loc[i]['GaiaID'])
    plx = np.append(plx,stuff[5][0])
    
scocen_stars['parallax'] = plx

In [226]:
ruwe_frac = scocen_stars['RUWE']/np.max(scocen_stars['RUWE'])
prob_frac = scocen_stars['MembershipProb']/np.max(scocen_stars['MembershipProb'])
chisq_frac = scocen_stars['chisq']/np.max(scocen_stars['chisq'])
for i in range(66):
    print('RUWE',1-ruwe_frac[i],scocen_stars['RUWE'].loc[i])
    print('MembProb',prob_frac[i],scocen_stars['MembershipProb'].loc[i])
    print('Chisq',chisq_frac[i],scocen_stars['chisq'].loc[i])
    score = 1-ruwe_frac[i]*prob_frac[i]*chisq_frac[i]
    print(score)
    print()

RUWE 0.927138284564425 1.0061806440353394
MembProb 0.9247311827956989 86
Chisq 1.0 1973.0
0.9326224997047372

RUWE 0.8748542153651342 1.7281951904296875
MembProb 0.8279569892473119 77
Chisq 0.8352762290927521 1648.0
0.9134525803343747

RUWE 0.9311277016427325 0.9510889649391174
MembProb 0.8709677419354839 81
Chisq 0.7080587937151546 1397.0
0.9575267036976968

RUWE 0.9332322644462836 0.9220260977745056
MembProb 0.7956989247311828 74
Chisq 0.5296502787633046 1045.0
0.9718612614905041

RUWE 0.9095032987658236 1.249710202217102
MembProb 0.9354838709677419 87
Chisq 0.5179929042067917 1022.0
0.956147650848086

RUWE 0.8922062729657415 1.4885727167129517
MembProb 0.956989247311828 89
Chisq 0.5174860618347694 1021.0
0.9466174638159451

RUWE 0.9277862786203346 0.99723219871521
MembProb 0.7849462365591398 73
Chisq 0.4419665484034465 872.0
0.9749476173059748

RUWE 0.9215227015198065 1.0837287902832031
MembProb 0.6236559139784946 58
Chisq 0.4262544348707552 841.0
0.979137901098884

RUWE 0.929708545

In [260]:
# Looking for companions of same parallax in Gaia:
from IPython.display import clear_output
from astroquery.gaia import Gaia
for i in np.arange(40,66,1):
    print(i,'HIP',scocen_stars['HipID'][i], 'GaiaID', scocen_stars['GaiaID'][i])
    string = "SELECT * FROM gaiadr2.gaia_source WHERE 1=CONTAINS(POINT('ICRS', ra, dec), CIRCLE('ICRS', "+str(scocen_stars['GaiaRA'][i])+","+str(scocen_stars['GaiaDEC'][i])+", 0.0083))"
    job = Gaia.launch_job(string)
    j = job.get_results()
    for jj in range(len(j)):
        print(j['source_id'][jj],j['parallax'][jj])
    input()
    clear_output()

In [263]:
print(np.where(scocen_stars['HipID']==80896)[0])

[0]


In [286]:
# Drop ones with close objects at similar parallax:
scocen_stars = scocen_stars.drop(np.where(scocen_stars['HipID']==80896)[0])
scocen_stars = scocen_stars.reset_index(drop=True)
print(np.where(scocen_stars['HipID']==55899)[0])
scocen_stars = scocen_stars.drop(np.where(scocen_stars['HipID']==55899)[0])
scocen_stars = scocen_stars.reset_index(drop=True)
print(np.where(scocen_stars['HipID']==55899)[0])
print(scocen_stars.shape)
scocen_stars

[24]
[]
(64, 11)


,HipID,MembershipProb,GaiaID,chisq,RUWE,AstrExNoise,GaiaRA,GaiaDEC,GaiaGMag,otype,parallax
0,62026,77,6073279508288863744,1648.0,1.728195,0.505047,190.707080,-55.947042,6.030624,b'Star',8.907770
1,56963,81,5370222361543311744,1397.0,0.951089,0.000000,175.160539,-49.509227,7.808117,b'Star',7.579360
2,64322,74,5863295048178769792,1045.0,0.922026,0.000000,197.745699,-62.087787,8.126640,b'Star',9.764732
3,76084,87,6208562250724337152,1022.0,1.249710,0.087385,233.083793,-31.142841,8.501759,b'Star',10.307512
4,67957,89,6090407734775872000,1021.0,1.488573,0.137400,208.755192,-50.750674,9.066534,b'Star',7.456648
5,78754,73,5998514882260320640,872.0,0.997232,0.131773,241.176287,-37.816228,7.035934,b'Star',5.810547
6,55334,58,5228361962579665152,841.0,1.083729,0.000000,169.969254,-70.618413,8.038264,b'Star',10.775763
7,61087,88,6054752668651631872,800.0,0.970687,0.000000,187.802322,-61.908794,7.861641,b'Star',9.257512
8,68722,72,6120528168618891904,638.0,1.234152,0.123006,211.020420,-37.283635,8.289087,b'Star',4.819026
9,68532,89,6111335598580052736,605.0,1.093626,0.079603,210.440239,-39.438720,7.512680,b'Star',9.011496


In [288]:
# Continue to drop ones with similar parallax neighbors:
scocen_stars = scocen_stars.drop(np.where(scocen_stars['HipID']==79690)[0])
scocen_stars = scocen_stars.reset_index(drop=True)
scocen_stars = scocen_stars.drop(np.where(scocen_stars['HipID']==71724)[0])
scocen_stars = scocen_stars.reset_index(drop=True)
scocen_stars = scocen_stars.drop(np.where(scocen_stars['HipID']==80062)[0])
scocen_stars = scocen_stars.reset_index(drop=True)
scocen_stars = scocen_stars.drop(np.where(scocen_stars['HipID']==59505)[0])
scocen_stars = scocen_stars.reset_index(drop=True)
scocen_stars = scocen_stars.drop(np.where(scocen_stars['HipID']==58899)[0])
scocen_stars = scocen_stars.reset_index(drop=True)
scocen_stars = scocen_stars.drop(np.where(scocen_stars['HipID']==79031)[0])
scocen_stars = scocen_stars.reset_index(drop=True)
scocen_stars = scocen_stars.drop(np.where(scocen_stars['HipID']==58283)[0])
scocen_stars = scocen_stars.reset_index(drop=True)
# Drop ones with parallax < 5 or > 10:
scocen_stars = scocen_stars.drop(np.where(scocen_stars['HipID']==68722)[0])
scocen_stars = scocen_stars.reset_index(drop=True)
scocen_stars = scocen_stars.drop(np.where(scocen_stars['HipID']==67919)[0])
scocen_stars = scocen_stars.reset_index(drop=True)
scocen_stars = scocen_stars.drop(np.where(scocen_stars['HipID']==63041)[0])
scocen_stars = scocen_stars.reset_index(drop=True)
scocen_stars = scocen_stars.drop(np.where(scocen_stars['HipID']==76084)[0])
scocen_stars = scocen_stars.reset_index(drop=True)
scocen_stars = scocen_stars.drop(np.where(scocen_stars['HipID']==72149)[0])
scocen_stars = scocen_stars.reset_index(drop=True)
scocen_stars = scocen_stars.drop(np.where(scocen_stars['HipID']==79860)[0])
scocen_stars = scocen_stars.reset_index(drop=True)
scocen_stars = scocen_stars.drop(np.where(scocen_stars['HipID']==59449)[0])
scocen_stars = scocen_stars.reset_index(drop=True)
scocen_stars = scocen_stars.drop(np.where(scocen_stars['HipID']==60009)[0])
scocen_stars = scocen_stars.reset_index(drop=True)

In [289]:
scocen_stars = scocen_stars.sort_values('MembershipProb',ascending=False)
scocen_stars = scocen_stars.reset_index(drop=True)
scocen_stars

,HipID,MembershipProb,GaiaID,chisq,RUWE,AstrExNoise,GaiaRA,GaiaDEC,GaiaGMag,otype,parallax
0,72627,93,6100404013458686592,245.0,1.083324,0.159724,222.744493,-42.822618,6.796734,b'Star',6.867985
1,71321,92,6099886349638035456,282.0,1.036709,0.000000,218.772141,-43.554645,7.784425,b'Star',7.877815
2,78196,90,6039491481598054528,66.0,1.044469,0.147173,239.497198,-31.729040,7.008297,b'Star',6.941908
3,68532,89,6111335598580052736,605.0,1.093626,0.079603,210.440239,-39.438720,7.512680,b'Star',9.011496
4,76633,89,6253938511486847360,16.0,0.950157,0.091894,234.750173,-19.732635,7.613636,b'Star',6.193137
5,66454,89,6084002083108541952,30.0,1.168738,0.415616,204.347646,-46.427969,5.879077,b'Star',7.072856
6,67957,89,6090407734775872000,1021.0,1.488573,0.137400,208.755192,-50.750674,9.066534,b'Star',7.456648
7,61087,88,6054752668651631872,800.0,0.970687,0.000000,187.802322,-61.908794,7.861641,b'Star',9.257512
8,60459,88,6053693083047384576,168.0,1.322249,0.251219,185.925438,-63.870124,7.375116,b'Star',9.237172
9,73913,88,6200763071150235520,351.0,1.140766,0.084342,226.574628,-35.406293,8.627187,b'Star',6.397933


In [290]:
scocen_stars = scocen_stars.drop(np.where(scocen_stars['HipID']==66908)[0])
scocen_stars = scocen_stars.reset_index(drop=True)
scocen_stars = scocen_stars.drop(np.where(scocen_stars['HipID']==55334)[0])
scocen_stars = scocen_stars.reset_index(drop=True)

scocen_stars = scocen_stars.sort_values('MembershipProb',ascending=False)
scocen_stars = scocen_stars.reset_index(drop=True)
scocen_stars

,HipID,MembershipProb,GaiaID,chisq,RUWE,AstrExNoise,GaiaRA,GaiaDEC,GaiaGMag,otype,parallax
0,72627,93,6100404013458686592,245.0,1.083324,0.159724,222.744493,-42.822618,6.796734,b'Star',6.867985
1,71321,92,6099886349638035456,282.0,1.036709,0.000000,218.772141,-43.554645,7.784425,b'Star',7.877815
2,78196,90,6039491481598054528,66.0,1.044469,0.147173,239.497198,-31.729040,7.008297,b'Star',6.941908
3,68532,89,6111335598580052736,605.0,1.093626,0.079603,210.440239,-39.438720,7.512680,b'Star',9.011496
4,76633,89,6253938511486847360,16.0,0.950157,0.091894,234.750173,-19.732635,7.613636,b'Star',6.193137
5,66454,89,6084002083108541952,30.0,1.168738,0.415616,204.347646,-46.427969,5.879077,b'Star',7.072856
6,67957,89,6090407734775872000,1021.0,1.488573,0.137400,208.755192,-50.750674,9.066534,b'Star',7.456648
7,61087,88,6054752668651631872,800.0,0.970687,0.000000,187.802322,-61.908794,7.861641,b'Star',9.257512
8,60459,88,6053693083047384576,168.0,1.322249,0.251219,185.925438,-63.870124,7.375116,b'Star',9.237172
9,73913,88,6200763071150235520,351.0,1.140766,0.084342,226.574628,-35.406293,8.627187,b'Star',6.397933


In [292]:
scocen_stars = scocen_stars.reindex(columns=['HipID','GaiaRA','GaiaDEC','MembershipProb','chisq','GaiaID', 'RUWE', \
                                             'AstrExNoise','GaiaGMag','otype','parallax'])
scocen_stars

,HipID,GaiaRA,GaiaDEC,MembershipProb,chisq,GaiaID,RUWE,AstrExNoise,GaiaGMag,otype,parallax
0,72627,222.744493,-42.822618,93,245.0,6100404013458686592,1.083324,0.159724,6.796734,b'Star',6.867985
1,71321,218.772141,-43.554645,92,282.0,6099886349638035456,1.036709,0.000000,7.784425,b'Star',7.877815
2,78196,239.497198,-31.729040,90,66.0,6039491481598054528,1.044469,0.147173,7.008297,b'Star',6.941908
3,68532,210.440239,-39.438720,89,605.0,6111335598580052736,1.093626,0.079603,7.512680,b'Star',9.011496
4,76633,234.750173,-19.732635,89,16.0,6253938511486847360,0.950157,0.091894,7.613636,b'Star',6.193137
5,66454,204.347646,-46.427969,89,30.0,6084002083108541952,1.168738,0.415616,5.879077,b'Star',7.072856
6,67957,208.755192,-50.750674,89,1021.0,6090407734775872000,1.488573,0.137400,9.066534,b'Star',7.456648
7,61087,187.802322,-61.908794,88,800.0,6054752668651631872,0.970687,0.000000,7.861641,b'Star',9.257512
8,60459,185.925438,-63.870124,88,168.0,6053693083047384576,1.322249,0.251219,7.375116,b'Star',9.237172
9,73913,226.574628,-35.406293,88,351.0,6200763071150235520,1.140766,0.084342,8.627187,b'Star',6.397933


In [295]:
scocen_stars.to_latex('targetlist_master.tex', index=False)

In [298]:
scocen_stars_out = scocen_stars.drop(['GaiaID', 'RUWE','AstrExNoise','otype','parallax'],axis=1)
scocen_stars_out.to_latex('targetlist.tex', index=False)